# Extracting training data from the ODC

* **Products used:** 
[gm_s2_semiannual](https://explorer.digitalearth.africa/gm_s2_semiannual)


## Description
This notebook will extract training data over Southern Africa using geometries within a shapefile (or geojson). To do this, we rely on a custom `deafrica-sandbox-notebooks` function called `collect_training_data`, contained within the [deafrica_tools.classification](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks/blob/minty-fresh-sandbox/Tools/deafrica_tools/classification.py) script.

1. Import, and preview our training data contained in the file: `'data/ceo_td_polys.geojson'`
2. Extract training data from the datacube using a custom defined feature layer function that we can pass to `collect_training_data`. The training data function is stored in the python file `feature_layer_functions.py` - the functions are stored in a seperate file simply to keep this notebook tidy.

    - **The features used to create the cropland mask are as follows:**
        - For two seasons, January to June, and July to Decemeber:
            - A geomedian composite of nine Sentinel-2 spectral bands
            - Three measures of median absolute deviation
            - NDVI, MNDWI, and LAI
            - Cumulative Rainfall from CHIRPS
        - Slope from SRTM (not seasonal, obviously)
            
3. Separate the coordinate values in the returned training data from step 2, and export the coordinates as a text file.
4. Export the remaining training data (features other than coordinates) to disk as a text file for use in subsequent scripts

***

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

In [1]:
!pip install -e ../../production/dea_ml
!pip install git+https://github.com/digitalearthafrica/deafrica-sandbox-notebooks.git@minty-fresh-sandbox#subdirectory=Tools

Obtaining file:///home/jovyan/Git_dev/crop-mask/production/dea_ml
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Running setup.py develop for dea-ml
You should consider upgrading via the '/env/bin/python3 -m pip install --upgrade pip' command.
  Cloning https://github.com/digitalearthafrica/deafrica-sandbox-notebooks.git (to revision minty-fresh-sandbox) to /tmp/pip-req-build-9_gkn7e4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 11.5 MB 20.2 MB/s eta 0:00:01
  Created wheel for deafrica-tools: filename=deafrica_tools-0.1.0-py3-none-any.whl size=78332 sha256=bcaadb54925ffa3b35c89325ec1da757e2be197ab7314e15662ddd66ca8ae73b
  Stored in directory: /tmp/pip-ephem-wheel-cache-g72_uexl/wheels/ff/4a/05/e0edf634fc047593b79eb41fdefeda18297de612675a8d4173
Successfully built deafrica-tools
  Attempt

## Load packages


In [ ]:
%matplotlib inline

import os
import warnings
warnings.filterwarnings("ignore")

import datacube
import numpy as np
import xarray as xr
import subprocess as sp
import geopandas as gpd
from odc.io.cgroups import get_cpu_quota
from datacube.utils.geometry import assign_crs
from datacube.utils.rio import configure_s3_access
configure_s3_access(aws_unsigned=True, cloud_defaults=True)

from deafrica_tools.plotting import map_shapefile
from deafrica_tools.classification import collect_training_data 

#import the custom feature layer functions
from feature_layer_functions import gm_mads_two_seasons_training

%load_ext autoreload
%autoreload 2 

## Analysis parameters

* `path`: The path to the input shapefile from which we will extract training data.
* `field`: This is the name of column in your shapefile attribute table that contains the class labels. **The class labels must be integers**


In [ ]:
path = 'data/Southern_training_data_20210421.geojson'

output_suffix = '20210421'

field = 'Class'

### Automatically find the number of cpus

> **Note**: With supervised classification, its common to have many, many labelled geometries in the training data. `collect_training_data` can parallelize across the geometries in order to speed up the extracting of training data. Setting `ncpus>1` will automatically trigger the parallelization, however, its best to set `ncpus=1` to begin with to assist with debugging before triggering the parallelization. 

In [ ]:
ncpus=round(get_cpu_quota())
print('ncpus = '+str(ncpus))

## Load & preview polygon data

We can load and preview our input data shapefile using `geopandas`. The shapefile should contain a column with class labels (e.g. 'class'). These labels will be used to train our model. 

> Remember, the class labels **must** be represented by `integers`.


In [ ]:
# Load input data shapefile
input_data = gpd.read_file(path)

# Plot first five rows
input_data.head()

In [ ]:
# Plot training data in an interactive map
# map_shapefile(input_data, attribute=field)

Now, we can pass this shapefile to `collect_training_data`.  For each of the geometries in our shapefile we will extract features in accordance with the function `feature_layer_functions.gm_mads_two_seasons_training`. These will include:

For two seasons, January to June, and July to Decemeber:
- A geomedian composite of nine Sentinel-2 spectral bands
- Three measures of median absolute deviation
- NDVI, MNDWI, and LAI
- Cumulative Rainfall from the CHIRPS
- Slope from SRTM

First, we need to set up a few extra inputs for `collect_training_data` and the datacube.  See the function docs [here](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks/blob/03b7b41d5f6526ff3f33618f7a0b48c0d10a155f/Scripts/deafrica_classificationtools.py#L650) for more information on these parameters.



In [ ]:
#set up our inputs to collect_training_data
zonal_stats = 'median'
return_coords = True

# Set up the inputs for the ODC query
products = ['gm_s2_semiannual']
time = ('2019')
measurements = [
        "blue",
        "green",
        "red",
        "nir",
        "swir_1",
        "swir_2",
        "red_edge_1",
        "red_edge_2",
        "red_edge_3",
        "bcdev",
        "edev",
        "sdev"
]
resolution = (-20, 20)
output_crs = 'epsg:6933'

In [ ]:
#generate a new datacube query object
query = {
    'time': time,
    'measurements': measurements,
    'resolution': resolution,
    'output_crs': output_crs,
    'group_by' : 'solar_day',
    'resampling': 'bilinear'
}

## Extract training data

> Remember, if running this function for the first time, its advisable to set `ncpus=1` to assist with debugging before triggering the parallelization (which won't return errors if something is not working correctly).  You can also limit the number of polygons to run for the first time by passing in `gdf=input_data[0:5]`, for example.

In [ ]:
%%time
column_names, model_input = collect_training_data(
                                    gdf=input_data[0:4],
                                    products=products,
                                    dc_query=query,
                                    ncpus=1,#ncpus,
                                    return_coords=return_coords,
                                    field=field,
                                    zonal_stats=zonal_stats,
                                    custom_func=gm_mads_two_seasons_training
                                    )

In [ ]:
print(column_names)
print('')
print(np.array_str(model_input, precision=2, suppress_small=True))

## Seperate the coordinates

By setting `return_coords=True` in the `collect_training_data` function, our training data now has two extra columns called `x_coord` and `y_coord`.  We need to seperate these from our training dataset as they will not be used to train the machine learning model. Instead, these variables will be used to help conduct Spatial K-fold Cross validation (SKVC) in the notebook `3_Train_fit_evaluate_classifier`.  For more information on why this is important, see this [article](https://www.tandfonline.com/doi/abs/10.1080/13658816.2017.1346255?journalCode=tgis20).

In [ ]:
coordinates_filename = "results/training_data/southern_training_data_coordinates_"+output_suffix+".txt"

In [ ]:
coord_variables = ['x_coord', 'y_coord']
model_col_indices = [column_names.index(var_name) for var_name in coord_variables]

np.savetxt(coordinates_filename, model_input[:, model_col_indices])

## Export training data

Once we've collected all the training data we require, we can write the data to disk. This will allow us to import the data in the next step(s) of the workflow.


In [ ]:
#set the name and location of the output file
output_file = "results/training_data/southern_training_data_"+output_suffix+".txt"

In [ ]:
#grab all columns except the x-y coords
model_col_indices = [column_names.index(var_name) for var_name in column_names[0:-2]]
#Export files to disk
np.savetxt(output_file, model_input[:, model_col_indices], header=" ".join(column_names[0:-2]), fmt="%4f")

## Next steps

To continue working through the notebooks in this `Southern Africa Cropland Mask` workflow, go to the next notebook `2_Inspect_training_data.ipynb`.

1. **Extract_training_data (this notebook)** 
2. [Inspect_training_data](2_Inspect_training_data.ipynb)

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Last modified:** Dec 2020
